+ 五点Gauss-Legendre求积公式

  查表可以得到在$[-1,1]$上积分的格式如下：

  节点：$0,\pm\frac{1}{3}\sqrt{5-2\sqrt{\frac{10}{7}}},\pm\frac{1}{3}\sqrt{5+2\sqrt{\frac{10}{7}}}$

  权重：$\frac{128}{225},\frac{332+13\sqrt{70}}{900},\frac{332-13\sqrt{70}}{900}$

  实际使用时，我们先对$[a,b]$上的积分做变换：
  $$
  \begin{align*}
  \int_a^b f(x)\mathrm{d} x =\frac{b-a}{2}\int_{-1}^1 f\left(s\left(\frac{b-a}{2}\right)+\frac{b+a}{2}\right)\mathrm{d} s
  \end{align*}
  $$
+ Romberg积分算法
    
   Romberg积分按照下面几个过程进行：首先求出$T_0^{(0)} =\frac{h}{2}[f(a)+f(b)]$；然后求梯形值$T_0(\frac{b-a}{2^k})$，也就是按照递推公式$T_{2n}=\frac{1}{2}T_n+\frac{h}{2}\sum_{k=0}^{n-1}f(x_{k+1/2})$计算每个$T_0^{(k)}$；随后用Richardson外推公式：
   $$
   \begin{align*}
   T_m^{(k)} = \frac{4^m}{4^m-1}T_{m-1}^{(k+1)}-\frac{1}{4^m-1}T_{m-1}^{(k)},
   \end{align*}
   $$
   逐个计算可求出$T$表的第$k$行的每个元素$T_j^{(k-j)}$，$j=1,2,\cdots$.
   
   最后，依据$|T^{(0)}_k-T_{k-1}^{(0)}|<\varepsilon$，则终止计算，并$T_k^{(0)}\approx I$，否则$k+1\to k$继续计算.

In [33]:
import numpy as np

def func(x):
    return (1/(x**2))*np.sin(2*np.pi/x)

########5-point Gauss-Legendre quadrature########
points = [0, (1/3)*np.sqrt(5-2*np.sqrt(10/7)), -(1/3)*np.sqrt(5-2*np.sqrt(10/7)),\
             (1/3)*np.sqrt(5+2*np.sqrt(10/7)), -(1/3)*np.sqrt(5+2*np.sqrt(10/7))]
weights = [128/225, (322+13*np.sqrt(70))/900, (322+13*np.sqrt(70))/900, (322-13*np.sqrt(70))/900, (322-13*np.sqrt(70))/900]
def GL5(fun, a, b):
    fi = [((b-a)/2)*func(xi*((b-a)/2)+(b+a)/2) for xi in points]
    return np.array(fi)@np.array(weights)
complex_5GL = GL5(func, 1, 1.5)+GL5(func, 1.5, 2)+GL5(func, 2, 2.5)+GL5(func, 2.5, 3)
print("="*10+"5-point Gauss-Legendre"+"="*10)
print("quadrature = {}".format(complex_5GL))

########Romberg quadrature########
def Trap(fun, a, b, Iold, k):
    if k == 1:
        Inew = (fun(a)+fun(b))*(b-a)/2
    else:
        n = 2**(k-2)
        h = (b-a)/n 
        x = a+(h/2)
        sum_k = 0
        for i in range(n):
            sum_k = sum_k + fun(x)
            x = x + h
        Inew = (Iold+h*sum_k)/2
    return Inew

def Richardson(R,k):
    for i in range(k-1,0,-1):
        c = 2**(2*(k-i))
        R[i] = (c*R[i+1]-R[i])/(c-1) 
    return R

def romberg(f, a, b, eps=1e-7):
    T = {}
    k = 1
    T[1] = Trap(f,a,b,0.0,1)
    former_R = T[1]
    while True:
        k += 1
        T[k] = Trap(f,a,b,T[k-1],k)
        T = Richardson(T,k)
        if abs(T[1] - former_R) < eps:
            return T[1]
        former_R = T[1]
        
romberg_quad = romberg(func, 1, 3) 
print("="*10+"Romberg"+"="*10)
print("quadrature = {}".format(romberg_quad))

==========5-point Gauss-Legendre==========
quadrature = -0.2387323403436461
==========Romberg==========
quadrature = -0.23873241462162365
